# Job search and application

In [1]:
# import libraries

import numpy as np
import pandas as pd
import regex as re # to clean text
import os #why?
import time # to save csv to today's date
import requests
from bs4 import BeautifulSoup #to get data out of html or xml
from collections import Counter # to count keyword appearance

In [2]:
# read the text file. the urls have been copied using the "Copy All Urls" chrome extension
# while going through the saved searches results, with Cmd+Enter open the interesting jobs in a new tab 

with open('./urls.txt',"r") as urls:
    urls = urls.readlines() 

In [3]:
#agent to interact with web content

hdr = {'User-Agent': 'Mozilla/5.0'} 

In [4]:
# check the request is successful

res = []
for url in urls:
    res.append(requests.get(url,headers=hdr))

for i in range(len(res)):
    if(res[i].status_code) != 200:
        print(i,res[i].status_code)

# get the content of each url

soup = []
for i in range(len(res)):
    soup.append(BeautifulSoup(res[i].content, 'lxml'))
    # soup is a list of beautifulsoup objects

45 403
128 403
162 410


In [5]:
# append each job description to a list

jd = []
for i in range(len(soup)):
    
# VentureFizz
    if soup[i].find('div',{'class': 'views-field views-field-field-job-description'}):
        jd.append(soup[i].find('div',{'class': 'views-field views-field-field-job-description'}))
    
# Linkedin
    elif soup[i].find('div',{'class': 'description__text description__text--rich'}):
        jd.append(soup[i].find('div',{'class': 'description__text description__text--rich'}))
    
# indeed.com
    elif soup[i].find('div',{'class': 'jobsearch-JobComponent icl-u-xs-mt--sm'}):
        jd.append(soup[i].find('div',{'class': 'jobsearch-JobComponent icl-u-xs-mt--sm'}))

# Glassdoor?
    elif soup[i].find(name='div', attrs={'id':'JobDescriptionContainer','class': 'gdGrid tabSection p-std mt-0'}):
        jd.append(soup[i].find(name='div', attrs={'id':'JobDescriptionContainer','class': 'gdGrid tabSection p-std mt-0'})) #glassdoor

# dice.com 
    else: 
        jd.append(soup[i].find(name='div', attrs={'id':'jobdescSec','class': 'highlight-black'}))

## Improvement point: 
- skip problematic jobs/lines

In [6]:
# split the job descriptions into a list of words
# if the job description can't be read(empty) let me know which post ...
# ... it has probably expired and needs to be ommitted

job_desc = [] # list of all words in the job desc
for i in range(len(jd)):
    try:
        job_desc.append(re.findall(r'[A-z]+', jd[i].text))
    except AttributeError:
        print('error on {}'.format (i))
        job_desc.append(re.findall(r'[A-z]+', jd[i-1].text)) # temp fix: if there is an error append 
                                                             # the prior cleared post
                                                            # code will break if two consecutive posts are problematic
    except AttributeError:
        print('error on {}'.format (i))
        job_desc.append(re.findall(r'[A-z]+', jd[i-2].text))
    except AttributeError:
        print('error on {}'.format (i))
        job_desc.append(re.findall(r'[A-z]+', jd[i-3].text))
    except AttributeError:
        print('error on {}'.format (i))
        job_desc.append(re.findall(r'[A-z]+', jd[i-4].text))

error on 12
error on 17
error on 26
error on 45
error on 67
error on 69
error on 74
error on 76
error on 78
error on 108
error on 113
error on 117
error on 124
error on 126
error on 128
error on 153
error on 155
error on 162


In [7]:
# small cap all keywords to be matched with the dictionary of search words

for i in range(len(job_desc)):
    job_desc[i] = [x.lower() for x in job_desc[i]]

In [8]:
#assign a weight for each keyword

keywords = {
            'cfa':10,'arabic':10,'french':10,'returnship':10,'re-ignite':10,'relaunch':10,'boston':10,
            'mba':7,'python':7,
            'fp':5,'analytics':5,'analysis':5,'analyze':5,'analyzes':5,'analytical':5,'analyzing':5,
            'tableau':5,'sql':5,'excel':5,
            'finance':3,'financial':3,'financials':3,
            'data':2,'dataset':2,'datasets':2,
            'predict':2, 'projections':2,'projections':2,'predictive':2, 'forecast':2, 'forecasting':2, 'forecasts':2,
            'valuation':2, 'valuations':2, 
            'model':2, 'models':2, 'modeling':2, 'decisions':2,
            'trend':1, 'trends':1, 'plan':1,'plans':1, 'planning':1, 'budget':1,
            'metric':1,'insight':1,'insights':1,
           }

In [9]:
# match the job description to the dictionary of keywords

keywords_lol = [] # list of (lists of) matching keywords of all jobs

for i in range(len(job_desc)):
    job_desc_keywords = [] # list of matching keywords for each job
    for word in job_desc[i]:
        if word in keywords.keys():
            job_desc_keywords.append(word)
    keywords_lol.append(job_desc_keywords)

In [10]:
# replace each keyword by its weight equivalent

scores_lol = [] # list of (lists of) job scores

for i in range(len(keywords_lol)):
    job_score = [] # score of each job
    for word in keywords_lol[i]:
        job_score.append(keywords[word])
    scores_lol.append(sum(job_score))

In [11]:
# count the times each keyword repeats in each job post
d = []
for i in range(len(keywords_lol)):
    cnt = Counter()
    for word in keywords_lol[i]:
        cnt[word] += 1
    d.append(cnt)
    #print(i,cnt)

In [12]:
# create the dataframe
jobs = pd.DataFrame(columns=['link', 'score'])

In [13]:
# clean up the urls
urls = [x.replace('\n',"") for x in urls]

In [14]:
# fill in the data frame
jobs = jobs.assign(score=scores_lol, link = urls).fillna(0)

In [15]:
# drop duplicate URLs
jobs = jobs.drop_duplicates(subset=['link','score'])

In [16]:
# create another column and mark all jobs as not applied to
jobs['status'] = 'not yet applied'

In [17]:
jobs.head()

,link,score,status
0,https://www.indeed.com/viewjob?jk=6c659eccabf4...,197,not yet applied
1,https://www.indeed.com/viewjob?jk=011b15460b87...,101,not yet applied
2,https://www.indeed.com/viewjob?jk=f83badafea23...,160,not yet applied
3,https://www.indeed.com/viewjob?jk=07f9fbc28997...,185,not yet applied
6,https://www.linkedin.com/jobs/view/1749285376/...,76,not yet applied


In [18]:
# do not abridge the urls
pd.set_option('display.max_colwidth', None)

In [19]:
# check the DF
jobs.head()

,link,score,status
0,"https://www.indeed.com/viewjob?jk=6c659eccabf4a45b&q=Python+Analytics+Data+Finance+MBA+excel+(analysis+or+Analyst+or+Analyses+or+Analytical+or+Analyzing+or+Dataset+or+Datasets+or+Financial+or+Financials+or+Decisions+or+Insight+or+Insights+or+Analytics+or+Analysis+or+Analyst+or+Analyses+or+Analytical+or+Anal&l=Brookline,+MA&tk=1e49ospml32vi802&from=ja&alid=5dcf23fc803d2451183c3f08&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1e49ospml32vi802",197,not yet applied
1,https://www.indeed.com/viewjob?jk=011b15460b8771f7&q=Python+Analytics+Data+Finance+MBA+excel+(analysis+or+Analyst+or+Analyses+or+Analytical+or+Analyzing+or+Dataset+or+Datasets+or+Financial+or+Financials+or+Decisions+or+Insight+or+Insights+or+Analytics+or+Analysis+or+Analyst+or+Analyses+or+Analytical+or+Anal&l=United+States&tk=1e3npd4df36v0800&from=ja&alid=5dcf2719803d2451183c4152&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1e3npd4df36v0800,101,not yet applied
2,https://www.indeed.com/viewjob?jk=f83badafea23ce6c&q=Python+Analytics+Data+Finance+MBA+excel+(analysis+or+Analyst+or+Analyses+or+Analytical+or+Analyzing+or+Dataset+or+Datasets+or+Financial+or+Financials+or+Decisions+or+Insight+or+Insights+or+Analytics+or+Analysis+or+Analyst+or+Analyses+or+Analytical+or+Anal&l=United+States&tk=1e4rrvhv02ta4800&from=ja&alid=5dcf2719803d2451183c4152&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1e4rrvhv02ta4800,160,not yet applied
3,https://www.indeed.com/viewjob?jk=07f9fbc2899778d1&q=python+analytics+data+finance+SQL+(analysis+or+analyze+or+analyses+or+analytical+or+analyzing+or+dataset+or+datasets+or+financial+or+financials+or+decisions+or+insight+or+insights+or+analytics+or+analysis+or+analyze+or+analyses+or+analytical+or+analyzing+&l=Brookline&tk=1e4rs41pi381h801&from=ja&advn=1343250976351098&adid=315897298&sjdu=i6xVERweJM_pVUvgf-MzuVJvpP-MaR_hWGGPPJrzQ5gblBxFjctZ0k8ChKLFXuLpEeQtSDbhkItY20XhSzlNdtEibdyPxFKZ-0PGAEnx3WY&acatk=1e4sf7dpahdmg800&pub=e4ea14867e765046&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails,185,not yet applied
6,https://www.linkedin.com/jobs/view/1749285376/?eBP=NotAvailableFromVoyagerAPI&refId=49160274-dcd6-4cfe-bc95-303d1c4fee85&trk=d_flagship3_search_srp_jobs,76,not yet applied


In [20]:
# save df to a csv file
jobs.to_csv('./jobs_{}.csv'.format(time.strftime("%Y%m%d")))

In [21]:
# show top 5 jobs
jobs.sort_values(['status', 'score'], ascending=[True, False] ).head()

,link,score,status
95,https://www.indeed.com/viewjob?jk=f51398bd7dd0474c&q=Python+Analytics+Data+Finance+MBA+excel+(analysis+or+Analyst+or+Analyses+or+Analytical+or+Analyzing+or+Dataset+or+Datasets+or+Financial+or+Financials+or+Decisions+or+Insight+or+Insights+or+Analytics+or+Analysis+or+Analyst+or+Analyses+or+Analytical+or+Anal&l=United+States&tk=1e5dr7617283p806&from=ja&advn=782336216336969&adid=329327108&sjdu=7BeRdT5uW8zd9J94pO4lsGI0Ja_kBs24iFGcI7wf15q550Mg1xAZesiZ5AabnRzOHq-Q0lbAQs6iicjzkHTf5wfanGPXGgjv-Gh8FMmu3qcMJJq_WWRfP6XhTeSU8Q4g&acatk=1e5h2dgsb4sv7800&pub=e4ea14867e765046&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails,298,not yet applied
20,https://www.linkedin.com/jobs/view/1806883960/?eBP=NotAvailableFromVoyagerAPI&refId=8837b68a-ed40-4b8b-9ead-5c2b2596e8e0&trk=d_flagship3_search_srp_jobs,242,not yet applied
64,https://www.linkedin.com/jobs/view/1806883960/?eBP=CwEAAAFxRS9HLsz6PcFWwrY6Soxli4AEf9V3sdQPqrDT2jSdQfMmSGpUI3ulV6E3awuBwPtDhB-YJyS3Zqz7uKw76z3qVRudYgVcSYOUrb68_os8q_D7T6zl7rT3n3E5Ckw0ZOIA9a0JV52MuSXGulsdByssu7-D3i2NvyE69qJdjjV34d4uxRvLWxkAmULvRqwMmfL3xPBGKvSzUVTNWRuzsyffElxaM1AY-wWfLXhdf4zoPyHI0EShRs1HuB0TXConUNSzRFzIKy9RjxpM_uIrglKT0tDcrh0YvqhiTf-om2caux0wU6Tvw387vRg-TfiP9_Tlz-K1eEuZ2UB3frz9vHy3zQJoz4uXLtrP3b0xdahPbxWPTzMHMZAgp1v4uHL2LIyygyerfg16JaU&refId=e2667841-283d-4c9a-ba7b-afaa25c979d8&trk=d_flagship3_search_srp_jobs,242,not yet applied
73,https://www.linkedin.com/jobs/view/1806883960/?eBP=JOB_SEARCH_ORGANIC&refId=7f1de094-9b9d-47cf-a3c4-7257d67a64fb&trk=d_flagship3_search_srp_jobs,242,not yet applied
74,https://www.linkedin.com/jobs/view/1741550750/?eBP=JOB_SEARCH_ORGANIC&recommendedFlavor=IN_NETWORK&refId=7f1de094-9b9d-47cf-a3c4-7257d67a64fb&trk=d_flagship3_search_srp_jobs,242,not yet applied


## Refresh point

In [28]:
# As you check the jobs, mark them as applied to ('yes') or do not qualify for ('x')

jobs.loc[[165,129,111,138,125,131,112,104,105,40,94,93,71,90,0,3,18,20,57,60,64,65,73,95],'status'] = 'yes'
jobs.loc[[190,99,168,153,152,134,140,173,141,176,171,167,166,169,108,174,162,175,147,126,155,154,113,158,149,
          137,135,103,106,133,122,118,121,77,70,62,14,6,88,84,72,74,89,87,91,82,78,83,1,2,11,12,13,19,21,
          24,26,27,28,31,48,61,66,67,68,69],'status'] = 'x'



In [25]:
# sort job by status status, then by score
top_job = jobs.sort_values(['status', 'score'], ascending=[True, False]).head(1)
top_job

,link,score,status
190,https://www.linkedin.com/jobs/view/financial-systems-analyst-fp-a-tableau-at-lyft-1744742229/?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic,177,not yet applied


In [26]:
# prior to applying, search the JD for required skills, experience, etc. # CMD + F
jd[top_job.index[0]].text.replace("\n", " ")


'At Lyft, our mission is to improve people’s lives with the world’s best transportation. To do this, we start with our own community by creating an open, inclusive, and diverse organization.Lyft is hiring a Financial Systems Analyst specializing in Tableau for its Financial Planning and Analysis Team. The candidate will work directly with the Finance Organization to build Tableau Dashboards that would steer Business Performance by connecting Data, People and Plans across Finance and Operations.Responsibilities Build key Tableau Dashboards that would steer Business Performance by Connecting Data, People and Plans Across Finance and Operations Lead FP&A team in Tableau reporting best practices, be a go-to expert for any Tableau dashboard requests Collaborate, support and advise business partners on strategic and financial aspects of the business with an eye to financial goals. You will work directly with Finance and Engineering functions Support forecasting, business planning and analysi

In [ ]:
#if you absolutely need to apply for a specific job post regardless of its score:
# myjob = jobs.loc[[128]] 
# jd[jobs.loc[[128]].index[0]].text

## Improvement points:
- show a more user friendly listing (maybe as it shows in the website)
- extract the parts of the text where the words experience, require, etc. is mentioned
- extract the city and job title (all this in a DF, maybe)

In [27]:
# check for keywords in top job ... to customize R/CL
pd.DataFrame.from_dict(d[top_job.index[0]], orient='index').sort_values(0, ascending=False)

,0
tableau,11
financial,8
finance,5
data,5
planning,4
analysis,3
sql,2
analytics,2
forecasting,2
fp,2


### back to refresh point